In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.autograd import Variable
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

from utils import *
import pickle

In [ ]:
df = pd.read_parquet("train.parquet")
df.head()

In [ ]:
n_gas = 2

In [ ]:
df1 = df[df.feature4 == f'gas{n_gas}']

In [ ]:
columns_for_predict = ['target0', 'target1']

### Model 2

In [ ]:
features = ['feature0',
 'feature1',
 'feature2',
 'feature3',
 'feature5',
 'feature6',
 'feature7',
 'feature8',
 'feature9',
 'feature10',
 'feature11',
 'feature12',
 'feature13',
 'feature14',
 'feature15',
 'feature16',
 'feature17',
 'feature18',
 'feature19',
 'feature20',
 'feature22',
 'feature23',
 'feature24']

In [ ]:
df = df1[df1['feature8'] > -30]

In [ ]:
X = df1[features]
y = df1[columns_for_predict]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=41)
X_train, y_train = X, y

In [ ]:
### normilize data

Scaler_x_train = StandardScaler()
Scaler_x_test = StandardScaler()
Scaler_y_train = StandardScaler()
Scaler_y_test = StandardScaler()


X_train = Scaler_x_train.fit_transform(X_train)
y_train  = Scaler_y_train.fit_transform(y_train)

X_test = Scaler_x_train.transform(X_test)
y_test  = Scaler_y_train.transform(y_test)

In [ ]:
# X_train = X_train.values
# y_train = y_train.values

# X_test = X_test.values
# y_test = y_test.values

In [ ]:
class Net(nn.Module):
    def __init__(self, input_size=24):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
#         self.fc21 = nn.Linear(128, 32)
        self.fc3 = nn.Linear(32, 2)
        
    def forward(self, x):
        x = torch.sigmoid(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
#         x = torch.sigmoid(self.fc21(x))
        x = self.fc3(x)
        return x


In [ ]:
batch_size = 128
n_epochs = 300
model = Net(input_size=X_test.shape[1])

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

In [ ]:
model, loss_values_train, loss_values_test = train(model, X_train, y_train, 
                                                   X_test, y_test, optimizer, criterion, batch_size, n_epochs)

In [ ]:
plt.plot(loss_values_train)
plt.plot(loss_values_test)
# plt.ylim((0,0.1))

#### test

In [ ]:
X_test_var = Variable(torch.FloatTensor(X_test), requires_grad=False) 
with torch.no_grad():
    test_result = model(X_test_var)

In [ ]:
# # inverse the y_test data back to 1,2,3,4
y_new_inverse = Scaler_y_train.inverse_transform(y_test)
y_pred_inverse = Scaler_y_train.inverse_transform(test_result)

mean_absolute_percentage_error(y_new_inverse, y_pred_inverse)

In [ ]:
# mean_absolute_percentage_error(y_test, test_result)

gas 2: 0.01196633358862598

gas 1: 0.02328757962759617


с нормализацией:

gas 1: 0.04109559396261545

gas 2: 0.01588126440058686


с логарифмом:

gas 1: 0.041498254378973407


In [ ]:
path = f'nn_gas{n_gas}_part2.pt'

In [ ]:
torch.save(model.state_dict(), path)

In [ ]:
pickle.dump(Scaler_x_train, open('scaler_x.pkl', 'wb'))
pickle.dump(Scaler_y_train, open('scaler_y.pkl', 'wb'))

# model 1

In [ ]:
features = ['feature0',
 'feature1',
 'feature2',
 'feature3',
 'feature5',
 'feature6',
 'feature7',
 'feature9',
 'feature10',
 'feature11',
 'feature12',
 'feature13',
 'feature14',
 'feature15',
 'feature16',
 'feature17',
 'feature18',
 'feature19',
 'feature20',
 'feature22',
 'feature23']

In [ ]:
class Net2(nn.Module):
    def __init__(self, input_size=24):
        super(Net2, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 2)
        
    def forward(self, x):
        x = F.sigmoid(self.fc1(x))
        x = F.sigmoid(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
df = df1[df1['feature8'] < -30]

In [ ]:
X = df[features]
y = df[columns_for_predict]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=41)
X_train, y_train = X, y

In [ ]:
X_train = X_train.values
y_train = y_train.values

X_test = X_test.values
y_test = y_test.values

In [ ]:
batch_size = 128
n_epochs = 400
model = Net(input_size=X_test.shape[1])

optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

model,loss_values_train, loss_values_test = train(model, X_train, y_train, X_test, 
                                                  y_test, optimizer, criterion, batch_size, n_epochs)

In [ ]:
plt.plot(loss_values_train)
plt.plot(loss_values_test)
# plt.ylim((0,0.1))

In [ ]:
loss_values_test

In [ ]:
X_test_var = Variable(torch.FloatTensor(X_test), requires_grad=False) 
with torch.no_grad():
    test_result = model(X_test_var)

In [ ]:
mean_absolute_percentage_error(y_test, test_result)

In [ ]:
path = f'nn_gas{n_gas}_part1.pt'

In [ ]:
torch.save(model.state_dict(), path)

Net2 0.015
Net 0.013 (10000)

In [ ]:
features